In [2]:
import pandas as pd

In [3]:
parks = pd.read_csv("national_park_trails.csv")

# Question 1
What is the average elevation gain of all trails in Alaska with a difficulty rating of 4 or higher?

In [7]:
parks[(parks["state_name"] == "Alaska") & (parks["difficulty_rating"] >= 4)][["elevation_gain"]].mean()

elevation_gain    1027.306629
dtype: float64

# Question 2
Which state has the most trails that are loop-type?

In [37]:
# loop trails is not in the correct df format, which is why idxmax() is producing "California"
# if you want it to convert to an actual df, do commented code
# the indexes in this are the states
loop_trails = parks[parks["route_type"] == "loop"].groupby("state_name")[["name"]].count()
loop_trails.idxmax()

# loop_trails = parks[parks["route_type"] == "loop"].groupby("state_name")[["name"]].count().reset_index()
# index = loop_trails["name"].idxmax()
# loop_trails["state_name"].iloc[index]

name    California
dtype: object

# Question 3
What is the total number of reviews for trails with the birding feature in their description?


In [60]:
parks[parks["activities"].apply(lambda x: "birding" in x)][["num_reviews"]].sum()


num_reviews    160501
dtype: int64

# Question 4
What is the average trail popularity for all trails in Yosemite National Park?


In [46]:
parks[parks["area_name"] == "Yosemite National Park"][["popularity"]].mean()

popularity    9.235686
dtype: float64

# Question 5
How many trails are categorized as both “hiking” and “birding” activities in California?


In [73]:
ca = parks[parks["state_name"] == "California"]
ca[ca["activities"].apply(lambda x: "birding" in x and "hiking" in x)].count()

trail_id             410
name                 410
area_name            410
city_name            410
state_name           410
country_name         410
_geoloc              410
popularity           410
length               410
elevation_gain       410
difficulty_rating    410
route_type           410
visitor_usage        394
avg_rating           410
num_reviews          410
features             410
activities           410
units                410
dtype: int64

# Question 6
What percentage of trails in the United States have a 5-star average rating?


In [80]:
trails_5 = parks[parks["avg_rating"] == 5][["trail_id"]].count()
trails_5
trails_total = parks["trail_id"].count()
trails_total
(trails_5/trails_total) *100

trail_id    16.269242
dtype: float64

# Question 7
Which city has the highest number of trails with the ‘waterfall’ feature?

In [87]:
waterfall = parks[parks["features"].apply(lambda x: "waterfall" in x)].groupby("city_name")[["trail_id"]].count()
waterfall.idxmax()

trail_id    Yosemite Valley
dtype: object

# Question 8
Find the trail with the highest difficulty rating in Denali National Park that also has a length shorter than 8 kilometers.

In [95]:
denali = parks[(parks["area_name"] == "Denali National Park") & (parks["length"] < 8000)]
denali.sort_values(by = "difficulty_rating", ascending = False).iloc[0]

trail_id                                                    10327620
name                                      Mount Thorofare Ridge Loop
area_name                                       Denali National Park
city_name                                                   Cantwell
state_name                                                    Alaska
country_name                                           United States
_geoloc                         {'lat': 63.43108, 'lng': -150.31096}
popularity                                                     5.799
length                                                      6598.294
elevation_gain                                              507.7968
difficulty_rating                                                  5
route_type                                                      loop
visitor_usage                                                    1.0
avg_rating                                                       4.5
num_reviews                       

# Question 9
How many trails have more than 50 reviews but a popularity score of less than 4?


In [98]:
parks[(parks["num_reviews"]> 50) & (parks["popularity"] < 4)].count()

trail_id             5
name                 5
area_name            5
city_name            5
state_name           5
country_name         5
_geoloc              5
popularity           5
length               5
elevation_gain       5
difficulty_rating    5
route_type           5
visitor_usage        5
avg_rating           5
num_reviews          5
features             5
activities           5
units                5
dtype: int64